# PydanticOutputParser

`PydanticOutputParser`는 언어 모델의 출력을 **구조화된 정보**로 변환하는 데 도움을 주는 클래스입니다. 이 클래스는 단순 텍스트 응답 대신 **명확하고 체계적인 형태로 필요한 정보를 제공**할 수 있습니다.

이 클래스를 활용하면 언어 모델의 출력을 특정 데이터 모델에 맞게 변환하여 정보를 더 쉽게 처리하고 활용할 수 있습니다.

## 주요 메서드

`PydanticOutputParser` (대부분의 OutputParser에 해당)에는 주로 **두 가지 핵심 메서드**가 구현되어야 합니다.

- **`get_format_instructions()`**: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공합니다. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 이 지침은 언어 모델이 출력을 구조화하고 특정 데이터 모델에 맞게 변환하는 데 매우 중요합니다.
- **`parse()`**: 언어 모델의 출력(문자열로 가정)을 받아 이를 특정 구조로 분석하고 변환합니다. Pydantic과 같은 도구를 사용하여 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다.

## 참고 자료

- [Pydantic 공식 도큐먼트](https://docs.pydantic.dev/latest/)


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

다음은 이메일 본문 예시입니다.


In [5]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

출력 파서를 사용하지 않는 경우 예시

In [7]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}",
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
chain=prompt | llm
answer = chain.stream({"email_conversation" : email_conversation})
output = stream_response(answer, return_output=True)

이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무는 바이크코퍼레이션의 대표로, 이은채 대리에게 연락을 취했습니다.
2. 바이크코퍼레이션은 "ZENESIS" 자전거에 관심이 있으며, 유통 협력을 제안하고 있습니다.
3. ZENESIS 모델의 상세한 브로슈어, 특히 기술 사양, 배터리 성능, 디자인 정보 요청.
4. 협력 논의를 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.
5. 미팅 장소는 이은채 대리의 사무실로 제안되었습니다.

위와 같은 이메일 내용이 주어졌을 때 아래의 Pydantic 스타일로 정의된 클래스를 사용하여 이메일의 정보를 파싱해 보겠습니다.

참고로, Field 안에 `description` 은 텍스트 형태의 답변에서 주요 정보를 추출하기 위한 설명입니다. LLM 이 바로 이 설명을 보고 필요한 정보를 추출하게 됩니다. 그러므로 이 설명은 정확하고 명확해야 합니다.

In [8]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")
    
# parser
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [9]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


프롬프트를 정의합니다.

1. `question`: 유저의 질문을 받습니다.
2. `email_conversation`: 이메일 본문의 내용을 입력합니다.
3. `format`: 형식을 지정합니다.


In [12]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT: # 여기에 출력 파서 포맷이 들어가게됨
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

다음으로는 Chain 을 생성합니다.


In [13]:
# chain 을 생성합니다.
chain = prompt | llm

체인을 실행하고 결과를 확인합니다.


In [14]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)

```json
{
    "person": "김철수",
    "email": "chulsoo.kim@bikecorporation.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 상세 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안합니다.",
    "date": "1월 15일 화요일 오전 10시"
}
```

마지막으로 `parser`를 사용하여 결과를 파싱하고 `EmailSummary` 객체로 변환합니다.


In [15]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(output)
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 상세 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안합니다.' date='1월 15일 화요일 오전 10시'


## parser 가 추가된 체인 생성

출력 결과를 정의한 Pydantic 객체로 생성할 수 있습니다.

In [16]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser

In [17]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션의 자전거 제조 및 유통 경험을 바탕으로 ZENESIS 모델의 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 정보가 필요하다고 언급했습니다. 또한, 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.', date='1월 15일 화요일 오전 10시')

## with_structured_output()

`.with_structured_output(Pydantic)`을 사용하여 출력 파서를 추가하면, 출력을 Pydantic 객체로 변환할 수 있습니다.

In [ ]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o"
).with_structured_output(EmailSummary)
# 이걸 쓰면 굳이 파서 지정안해도 가능하다는게 장점인거겠지?

In [19]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 상세 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안합니다.', date='1월 15일 화요일 오전 10시')

**참고**

한 가지 아쉬운 점은 `.with_structured_output()` 함수는 `stream()` 기능을 지원하지 않습니다.

# CommaSeparatedListOutputParser

`CommaSeparatedListOutputParser`는 쉼표로 구분된 항목 목록을 반환할 필요가 있을 때 유용한 출력 파서입니다.

이 파서를 사용하면, 입력된 데이터나 요청된 정보를 쉼표로 구분하여 명확하고 간결한 목록 형태로 제공할 수 있습니다. 예를 들어, 여러 개의 데이터 포인트, 이름, 항목 또는 다양한 값을 나열할 때 효과적으로 정보를 정리하고 사용자에게 전달할 수 있습니다.

이 방법은 정보를 구조화하고 가독성을 높이며, 특히 데이터를 다루거나 리스트 형태의 결과를 요구하는 경우에 매우 유용합니다.

In [23]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# print(format_instructions)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

chain.invoke({"subject": "과일"})

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


['사과', '바나나', '딸기', '수박', '포도']

In [24]:
for s in chain.stream({"subject": "과일"}):
    print(s)

['사과']
['바나나']
['딸기']
['수박']
['포도']


# StructuredOutputParser

StructuredOutputParser는 LLM에 대한 답변을 `dict` 형식으로 구성하고, key/value 쌍으로 여러 필드를 반환하고자 할 때 유용하게 사용할 수 있습니다. 

## 장점
Pydantic/JSON 파서가 더 강력하다는 평가를 받지만, StructuredOutputParser는 로컬 모델과 같은 덜 강력한 모델에서도 유용합니다. 이는 GPT나 Claude 모델보다 인텔리전스가 낮은(즉, parameter 수가 적은) 모델에서 특히 효과적입니다. 

## 참고 사항
로컬 모델의 경우 `Pydantic` 파서가 동작하지 않는 상황이 빈번하게 발생할 수 있습니다. 이러한 경우, 대안으로 StructuredOutputParser를 사용하는 것이 좋은 해결책이 될 수 있습니다.


In [28]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
# response schema는 출력의 구조화하는데 사용됨..아마도..abs

response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
    
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template = "answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)
chain=prompt | model | output_parser

chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '서울', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

# JsonOutputParser

JsonOutputParser는 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 도구입니다. 이 도구는 Large Language Model (LLM)이 데이터를 조회하고 결과를 도출할 때, 지정된 스키마에 맞게 JSON 형식으로 데이터를 반환할 수 있도록 설계되었습니다.

LLM이 데이터를 정확하고 효율적으로 처리하여 사용자가 원하는 형태의 JSON을 생성하기 위해서는, 모델의 용량(예: 인텔리전스)이 충분히 커야 합니다. 예를 들어, llama-70B 모델은 llama-8B 모델보다 더 큰 용량을 가지고 있어 보다 복잡한 데이터를 처리하는 데 유리합니다.

**[참고]**

`JSON (JavaScript Object Notation)` 은 데이터를 저장하고 구조적으로 전달하기 위해 사용되는 경량의 데이터 교환 포맷입니다. 웹 개발에서 매우 중요한 역할을 하며, 서버와 클라이언트 간의 통신을 위해 널리 사용됩니다. JSON은 읽기 쉽고, 기계가 파싱하고 생성하기 쉬운 텍스트를 기반으로 합니다.

JSON의 기본 구조
JSON 데이터는 이름(키)과 값의 쌍으로 이루어져 있습니다. 여기서 "이름"은 문자열이고, "값"은 다양한 데이터 유형일 수 있습니다. JSON은 두 가지 기본 구조를 가집니다:

- 객체: 중괄호 {}로 둘러싸인 키-값 쌍의 집합입니다. 각 키는 콜론 :을 사용하여 해당하는 값과 연결되며, 여러 키-값 쌍은 쉼표 ,로 구분됩니다.
- 배열: 대괄호 []로 둘러싸인 값의 순서 있는 목록입니다. 배열 내의 값은 쉼표 ,로 구분됩니다.

```json
{
  "name": "John Doe",
  "age": 30,
  "is_student": false,
  "skills": ["Java", "Python", "JavaScript"],
  "address": {
    "street": "123 Main St",
    "city": "Anytown"
  }
}
```

In [7]:
# Q. jsonoutput이랑 pydantic, structuredoutputparser의 차이점은 무엇인가요?
# A. jsonoutput은 출력을 JSON 형식으로 반환하는데 사용되며, Pydantic은 Pydantic 모델로 파싱하는데 사용됩니다. StructuredOutputParser는 출력을 구조화하는데 사용됩니다.

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")
    
question = "민주주의 중요성에 대해 설명해주세요"

parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "title": "Description", "type": "string"}, "hashtags": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절하 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요"),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions = parser.get_format_instructions())

chain = prompt | model | parser
answer = chain.invoke({"question" : question})
print(type(answer))
print(answer)

<class 'dict'>
{'description': '민주주의는 국민이 주권을 가지고 정치적 의사 결정에 참여할 수 있는 체제를 의미합니다. 이는 개인의 자유와 권리를 보장하고, 다양한 의견을 수렴하여 사회적 합의를 도출하는 데 중요한 역할을 합니다. 민주주의는 또한 권력의 남용을 방지하고, 투명성과 책임성을 증진시킴으로써 사회의 안정과 발전을 도모합니다.', 'hashtags': '#민주주의 #자유 #권리 #참여'}


**Pydantic 을 사용하지 않고 `JsonOutputParser` 를 사용**

Pydantic 없이도 이 기능을 사용할 수 있습니다. 이 경우 JSON을 반환하도록 요청하지만, 스키마가 어떻게 되어야 하는지에 대한 구체적인 정보는 제공하지 않습니다.

In [10]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)

{'description': '지구 온난화는 대기 중 온실가스 농도의 증가로 인해 지구의 평균 기온이 상승하는 현상입니다. 이는 주로 화석 연료의 연소, 산림 파괴, 산업 활동 등 인간의 활동에 의해 촉발됩니다. 지구 온난화는 극지방의 빙하 감소, 해수면 상승, 기후 패턴 변화 등 다양한 환경적 영향을 초래하며, 생태계와 인간 사회에 심각한 영향을 미칠 수 있습니다.', 'hashtags': ['#지구온난화', '#온실가스', '#기후변화', '#환경보호', '#지속가능성']}


In [ ]:
print(parser.get_format_instructions()) 
# pydantic을 사용하면 아래보다 더 구체적인 정보를 제공함

Return a JSON object.


# PandasDataFrameOutputParser

**Pandas DataFrame**은 Python 프로그래밍 언어에서 널리 사용되는 데이터 구조로, 데이터 조작 및 분석을 위한 강력한 도구입니다. DataFrame은 구조화된 데이터를 효과적으로 다루기 위한 포괄적인 도구 세트를 제공하며, 이를 통해 데이터 정제, 변환 및 분석과 같은 다양한 작업을 수행할 수 있습니다.

이 **출력 파서**는 사용자가 임의의 Pandas DataFrame을 지정하여 해당 DataFrame에서 데이터를 추출하고, 이를 형식화된 사전(dictionary) 형태로 조회할 수 있게 해주는 LLM(Large Language Model) 기반 도구입니다.


In [14]:
import pprint
from typing import Any, Dict
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [26]:
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [27]:
df = pd.read_csv(r'C:\Users\SONG\Desktop\GIT\langchain-kr\03-OutputParser\data\titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
parser = PandasDataFrameOutputParser(dataframe=df)
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [29]:
# 열 작업 예시입니다.
df_query = "Age column 을 조회해 주세요."


# 프롬프트 템플릿을 설정합니다.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{question}\n",
    input_variables=["question"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 출력
format_parser_output(parser_output)

{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0,
         5: nan,
         6: 54.0,
         7: 2.0,
         8: 27.0,
         9: 14.0,
         10: 4.0,
         11: 58.0,
         12: 20.0,
         13: 39.0,
         14: 14.0,
         15: 55.0,
         16: 2.0,
         17: nan,
         18: 31.0,
         19: nan}}


In [30]:
df_query = "Retrieve the first row"

parser_output = chain.invoke({"question": df_query})

format_parser_output(parser_output)

{'0': {'Age': 22.0,
       'Cabin': nan,
       'Embarked': 'S',
       'Fare': 7.25,
       'Name': 'Braund, '
               'Mr. '
               'Owen '
               'Harris',
       'Parch': 0,
       'PassengerId': 1,
       'Pclass': 3,
       'Sex': 'male',
       'SibSp': 1,
       'Survived': 0,
       'Ticket': 'A/5 '
                 '21171'}}


In [31]:
# row 0-4의 평균 나이를 구합니다

df["Age"].head().mean()

31.2

In [33]:
# 임의의 Pandas DataFrame 작업 예시, 행의 수를 제한합니다.
df_query = "Retrieve the average of the Ages from row 0 to 4."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

{'mean': 31.2}


In [35]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# 잘못 형식화된 쿼리의 예시입니다. --> ?? 뭐가 잘못됐단건지 이해안됨
df_query = "Calculate average `Fare` rate."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

print(df["Fare"].mean())

{'mean': 22.19937}
22.19937


# DatetimeOutputParser

`DatetimeOutputParser` 는 LLM의 출력을 `datetime` 형식으로 파싱하는 데 사용할 수 있습니다.


**참고**

| 형식 코드 | 설명                | 예시          |
|------------|---------------------|---------------|
| %Y         | 4자리 연도          | 2024          |
| %y         | 2자리 연도          | 24            |
| %m         | 2자리 월            | 07            |
| %d         | 2자리 일            | 04            |
| %H         | 24시간제 시간       | 14            |
| %I         | 12시간제 시간       | 02            |
| %p         | AM 또는 PM          | PM            |
| %M         | 2자리 분            | 45            |
| %S         | 2자리 초            | 08            |
| %f         | 마이크로초 (6자리)  | 000123        |
| %z         | UTC 오프셋          | +0900         |
| %Z         | 시간대 이름         | KST           |
| %a         | 요일 약어           | Thu           |
| %A         | 요일 전체           | Thursday      |
| %b         | 월 약어             | Jul           |
| %B         | 월 전체             | July          |
| %c         | 전체 날짜와 시간     | Thu Jul  4 14:45:08 2024 |
| %x         | 전체 날짜           | 07/04/24      |
| %X         | 전체 시간           | 14:45:08      |

In [36]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 날짜 및 시간 출력 파서
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

# 사용자 질문에 대한 답변 템플릿
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    },  # 지침을 템플릿에 적용
)

# 프롬프트 내용을 출력
prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 0689-04-09, 1354-10-11, 0863-07-14\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:')

In [38]:
# Chain 을 생성합니다.
chain = prompt | ChatOpenAI() | output_parser

# 체인을 호출하여 질문에 대한 답변을 받습니다.
output = chain.invoke({"question": "Google 이 창업한 연도"})
# 결과를 문자열로 변환
output.strftime("%Y-%m-%d")

'1998-09-04'

# EnumOutputParser

LangChain의 EnumOutputParser는 언어 모델의 출력을 미리 정의된 열거형(Enum) 값 중 하나로 파싱하는 도구입니다. 이 파서의 주요 특징과 사용법은 다음과 같습니다.

## 주요 특징

- **열거형 파싱**: 문자열 출력을 미리 정의된 Enum 값으로 변환합니다.
- **타입 안전성**: 파싱된 결과가 반드시 정의된 Enum 값 중 하나임을 보장합니다.
- **유연성**: 공백이나 줄바꿈 문자를 자동으로 처리합니다.

## 사용 방법

EnumOutputParser는 언어 모델의 출력에서 유효한 Enum 값을 추출하는 데 유용합니다. 이를 통해 출력 데이터의 일관성을 유지하고 예측 가능성을 높일 수 있습니다. 파서를 사용하려면, 미리 정의된 Enum 값을 설정하고 해당 값을 기준으로 문자열 출력을 파싱합니다.

In [40]:
from langchain.output_parsers.enum import EnumOutputParser

from enum import Enum 

class Color(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"
    
parser = EnumOutputParser(enum=Color)

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI 

prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?
    
Object: {object}

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())

chain = prompt | ChatOpenAI() | parser

response = chain.invoke({"object": "apple"})
print(response)

Color.RED


In [41]:
type(response)

<enum 'Color'>

In [42]:
response.value

'red'

# OutputFixingParser

`OutputFixingParser`는 출력 파싱 과정에서 발생할 수 있는 오류를 자동으로 수정하는 기능을 제공합니다. 이 파서는 기본적으로 다른 파서, 예를 들어 `PydanticOutputParser` 를 래핑하고, 이 파서가 처리할 수 없는 형식의 출력이나 오류를 반환할 경우, 추가적인 LLM 호출을 통해 오류를 수정하도록 설계되었습니다.

이러한 접근 방식의 핵심은, 첫 번째 시도에서 스키마를 준수하지 않는 결과가 나올 경우, `OutputFixingParser` 가 자동으로 형식이 잘못된 출력을 인식하고, 이를 수정하기 위한 새로운 명령어와 함께 모델에 다시 제출한다는 것입니다. 이 과정에서, 수정을 위한 명령어는 오류를 정확히 지적하고, 올바른 형식으로 데이터를 재구성할 수 있도록 구체적인 지시를 포함해야 합니다.

예를 들어, `PydanticOutputParser` 를 사용하여 특정 데이터 스키마를 준수하는 출력을 생성하려고 했지만, 일부 필드가 누락되었거나 데이터 유형이 잘못된 경우가 발생할 수 있습니다. 이때 `OutputFixingParser` 는 다음 단계로, 해당 오류를 수정하는 지시를 포함한 새로운 요청을 LLM에 제출합니다. LLM은 이 지시를 바탕으로 오류를 수정한 새로운 출력을 생성하게 됩니다.


In [43]:
from langchain_openai import ChatOpenAI 
from langchain.output_parsers import PydanticOutputParser 
from pydantic import BaseModel, Field 
from typing import List 

class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description = "list of names of films they starred in")

actor_query = "Generate the filmography for a random actor."
parser = PydanticOutputParser(pydantic_object=Actor)

# 잘못된 형식을 일부러 입력
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

parser.parse(misformatted)

OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}

In [45]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

print(misformatted)

actor = new_parser.parse(misformatted)

print(actor)

{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
name='Tom Hanks' film_names=['Forrest Gump']
